<a href="https://colab.research.google.com/github/marbortoli/GoogleColab/blob/main/TFOD_MASKRCNN_Marina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
import pathlib


In [20]:
# Clone tensorflow models repository
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1691106 sha256=aa1d878d5679c42c4fdc8439dd1c74d985522d503148c4618a1800cb80044a16
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vnm26_0/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


## import dataset with images and annotations (in train and test folders)

In [ ]:
# import dataset with images and annotations (in train and test folders)

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls gdrive/MyDrive/dataset


#dataset = "./gdrive/My Drive/dataset"
#!wget -q --show-progress 'https://drive.google.com/drive/folders/1ZGzVo2zeDqmSjyY2BbfBMJebELQAU_FN?usp=sharing'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
test  train


In [ ]:
# v7 database
!pip install darwin-py

In [21]:
# authenticate
import darwin

!darwin authenticate
# key yjoOEYG.Q9JUmWG2o7jfqQ-SV3eLlRnzrO01Y7Yr

ModuleNotFoundError: ignored

In [ ]:
# import images from v7
!darwin dataset pull energy-machines-cloud-object-detection/scada-hmi_01:exportdata0145

In [23]:
!python '/content/gdrive/MyDrive/coco_tf_record.py' \
  --train_image_dir='/content/gdrive/MyDrive/dataset/train/images/' \
  --test_image_dir='/content/gdrive/MyDrive/dataset/test/images/' \
  --train_annotations_file='/content/gdrive/MyDrive/dataset/train/annotations/train.json' \
  --test_annotations_file='/content/gdrive/MyDrive/dataset/test/annotations/test.json' \
  --logtostderr --output_dir='/content/./'

2022-04-26 15:55:17.239777: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
INFO:tensorflow:Found groundtruth annotations. Building annotations index.
I0426 15:55:17.727666 140133629024128 coco_tf_record.py:194] Found groundtruth annotations. Building annotations index.
INFO:tensorflow:0 images are missing annotations.
I0426 15:55:17.728476 140133629024128 coco_tf_record.py:207] 0 images are missing annotations.
INFO:tensorflow:On image 0 of 105
I0426 15:55:17.728808 140133629024128 coco_tf_record.py:212] On image 0 of 105
INFO:tensorflow:On image 100 of 105
I0426 15:55:52.171670 140133629024128 coco_tf_record.py:212] On image 100 of 105
INFO:tensorflow:Finished writing, skipped 0 annotations.
I0426 15:55:53.421161 140133629024128 coco_tf_record.py:219] Finished writing, skipped 0 annotations.
INFO:tensorflow:Found groundtruth annotations. Building annotations index.
I0426 15:55:53.723045 140133629024

In [25]:
# get ready labelmap
## The id number of each item should match the ids inside the train.json and test.json files.

!wget https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt

--2022-04-26 15:59:05--  https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘scada_label_map.pbtxt’

scada_label_map.pbt     [ <=>                ] 145.63K  --.-KB/s    in 0.04s   

2022-04-26 15:59:05 (3.83 MB/s) - ‘scada_label_map.pbtxt’ saved [149124]



In [27]:
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'
labelmap_path = '/content/scada_label_map.pbtxt'

## configure training

In [28]:
# model basic parameters configuration

batch_size = 4
num_steps = 3000
num_eval_steps = 1000

In [29]:
# select pretrained model from tensorflow model zoo
#  Tensorflow 2 Detection Model Zoo
## I want mask_rcnn to make use of the json quality

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
!tar -xf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

--2022-04-26 15:59:39--  http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.212.128, 2607:f8b0:400c:c11::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.212.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484546405 (462M) [application/x-tar]
Saving to: ‘mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz’

mask_rcnn_inception 100%[===================>] 462.10M   179MB/s    in 2.6s    

2022-04-26 15:59:42 (179 MB/s) - ‘mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz’ saved [484546405/484546405]



In [30]:
fine_tune_checkpoint = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'

In [31]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config

base_config_path = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config'

--2022-04-26 15:59:55--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3943 (3.9K) [text/plain]
Saving to: ‘mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config’

mask_rcnn_inception 100%[===================>]   3.85K  --.-KB/s    in 0s      

2022-04-26 15:59:56 (55.8 MB/s) - ‘mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config’ saved [3943/3943]



In [32]:
# edit configuration file

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(4), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [33]:
%cat model_config.config

# Mask R-CNN with Inception Resnet v2 (no atrous)
# Sync-trained on COCO (with 8 GPUs) with batch size 16 (1024x1024 resolution)
# Initialized from Imagenet classification checkpoint
#
# Train on GPU-8
#
# Achieves 40.4 box mAP and 35.5 mask mAP on COCO17 val

model {
  faster_rcnn {
    number_of_stages: 3
    num_classes: 4
    image_resizer {
      fixed_shape_resizer {
        height: 1024
        width: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_resnet_v2_keras'
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score

In [34]:
model_dir = 'training/'
pipeline_config_path = 'model_config.config'

## train custom model

In [41]:
!pip install opencv-python


In [43]:
!python '/content/models/research/object_detection/model_main_tf2.py' \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr

2022-04-26 16:08:16.855448: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0426 16:08:16.857519 139860695066496 cross_device_ops.py:1386] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0426 16:08:16.862684 139860695066496 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0426 16:08:16.866864 139860695066496 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0426 16:08:16.867100 139860695066496 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0426 16:08:17.066318 139860695066496 deprecation.py:343] From /usr/local/

In [ ]:
# visualize model efficiency 

%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

## export model inference graph

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
# download

from google.colab import files
files.download(f'/content/{output_directory}/saved_model/saved_model.pb') 